<a href="https://colab.research.google.com/github/YIFUNLIN/Autonomous_vehicle/blob/main/%E3%80%90Demo06b%E3%80%91RAG02_%E6%89%93%E9%80%A0_RAG_%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

資料集：1 0 2 年 度 公 民 營 汽 車 駕 駛 人
訓 練 機 構 小 型 車 駕 駛 教 練 訓 練 交通法規

In [9]:
URL = "https://drive.google.com/uc?export=download&id=1RtzdqVpoNbzSI0EnE-OZNxBjK3dEVtYE"

!wget -O faiss_db.zip "$URL"

--2025-04-20 15:46:06--  https://drive.google.com/uc?export=download&id=1RtzdqVpoNbzSI0EnE-OZNxBjK3dEVtYE
Resolving drive.google.com (drive.google.com)... 172.217.15.238, 2607:f8b0:4025:802::200e
Connecting to drive.google.com (drive.google.com)|172.217.15.238|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1RtzdqVpoNbzSI0EnE-OZNxBjK3dEVtYE&export=download [following]
--2025-04-20 15:46:06--  https://drive.usercontent.google.com/download?id=1RtzdqVpoNbzSI0EnE-OZNxBjK3dEVtYE&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.0.65, 2607:f8b0:4025:810::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.0.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8934 (8.7K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]   8.72K  --.-KB/s    in 0s      

2

In [10]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.pkl      
  inflating: faiss_db/index.faiss    


### 1. 安裝並引入必要套件

In [3]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [12]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [45]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 `faiss_db`

In [46]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 4. 設定好我們要的 LLM

In [47]:
import os
from google.colab import userdata

如之前, 我們會用 OpenAI API。這裡使用 Groq 服務, 可改成你要的服務。

In [48]:
api_key = userdata.get('Gemini')

In [39]:
from google import genai

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Hello"
)
print(response.text)

Hello! How can I help you today?



### 5. `prompt` 設計

In [49]:
system_prompt = "你是台灣交通法規的專家，請全程使用繁體中文回答問題"

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

請根據資料內容回覆，若資料不足請告訴同學可以請教他打電話去諮詢
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [40]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history


    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])


    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    response = client.models.generate_content(
    model="gemini-2.0-flash", contents=[
            {"role": "user", "parts": [f"{system_prompt}\n\n{final_prompt}"]}
        ]
    )

    print(response)
    answer = response.text
    print(answer)
    chat_history.append((user_input, answer))
    print(chat_history)
    return answer

In [51]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history

    # 🔍 1. 從向量資料庫取回相關文件
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 🧠 2. 自訂 Prompt（你需要先定義 prompt_template 和 system_prompt）
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    combined_prompt = f"{system_prompt}\n\n{final_prompt}"

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            {
                "role": "user",
                "parts": [{"text": combined_prompt}]
            }
        ]
    )

    if response.candidates and response.candidates[0].content.parts:
        answer = response.candidates[0].content.parts[0].text
    else:
        return "No valid response received from the API."

    chat_history.append((user_input, answer))
    print("Chat history:", chat_history)

    return answer


### 7. 用 Gradio 打造 Web App

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🎓 AI 交通法規大王")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-53-24b93c1ceed8>:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://be7d4f50276dcece9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Chat history: [('法規？', '根據你提供的資料，我看到以下幾點關於法規的資訊：\n\n1.  **年度：** 資料編製於民國102年10月，請注意法規可能已更新。\n2.  **主要法規類別：**\n    *   道路交通安全規則\n    *   道路交通管理處罰條例\n    *   高速公路及快速公路交通管制規則\n    *   道路交通標誌標線號誌設置規則\n3.  **行車執照換發：**\n    *   汽車行車執照、拖車使用證：每3年換發一次。\n    *   機車行車執照：每2年換發一次。\n    *   換發時間：自原發照之日起算，期滿前後1個月內。\n    *   未換發不得行駛。\n\n**重要提醒：**\n\n*   由於你提供的資料是民國102年的，交通法規經過多年可能已經有修改或更新。\n*   **最可靠的方式是查詢最新的法規版本。** 你可以透過以下方式查詢：\n\n    *   **交通部公路總局網站：** 這是查詢最新交通法規最直接的管道。\n    *   **全國法規資料庫：** 這個網站提供全國的法規資訊，可以搜尋相關的交通法規。\n\n**建議：**\n\n如果你有具體的交通法規問題，例如想要了解某個特定違規行為的罰則，或是關於某種車輛的特別規定，請提供更詳細的資訊，我才能更精準地回答。\n\n**如果以上資訊仍然無法解答你的疑問，建議你直接致電交通部公路總局或相關的交通監理單位，向他們詢問最準確的資訊。**\n'), ('法規？', '根據您提供的資料，我看到以下相關的交通法規：\n\n*   **道路交通安全規則** (資料中標示為"一、道路交通安全規則")\n*   **道路交通管理處罰條例** (資料中標示為"二、道路交通管理處罰條例")\n*   **高速公路及快速公路交通管制規則** (資料中標示為"三、高速公路及快速公路交通管制規則")\n*   **道路交通標誌標線號誌設置規則** (資料中標示為"四、道路交通標誌標線號誌設置規則")\n\n另外，關於行車執照的換發，資料中提到：\n\n*   **汽車行車執照、拖車使用證每3年換發1次**\n*   **機車行車執照每2年換發1次**\n*   **自原發照之日起算，期滿前後1個月內，須申請換領新照始得行駛。**\n\n如果您